In [1]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

from datetime import datetime

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"



In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


## Params

In [4]:
nfft = 512
hop_size = 256 

## Load csv

In [5]:
train_txt_path = '../dataset/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt'

df_train = pd.read_csv(train_txt_path, sep=" ", header=None)
df_train.columns = ["speaker_id", "audio_filename", "null", "system_id", "label"]
df_train = df_train.drop(columns="null")

In [6]:
dev_txt_path = '../dataset/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt'

df_dev = pd.read_csv(dev_txt_path, sep=" ", header=None)
df_dev.columns = ["speaker_id", "audio_filename", "null", "system_id", "label"]
df_dev = df_dev.drop(columns="null")

In [7]:
eval_txt_path = '../dataset/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt'

df_eval = pd.read_csv(eval_txt_path, sep=" ", header=None)
df_eval.columns = ["speaker_id", "audio_filename", "null", "system_id", "label"]
df_eval = df_eval.drop(columns="null")

In [8]:
partition = {} 

partition['train'] = df_train['audio_filename'].tolist()
partition['dev'] = df_dev['audio_filename'].tolist()
partition['eval'] = df_eval['audio_filename'].tolist()

### Extract labels

In [9]:
merged_df = df_train.append([df_eval, df_dev])

labels = dict(zip(merged_df.audio_filename, merged_df.label))

one_labels = {x : 1 for [x, y] in labels.items() if y !='bonafide'}  
zeros_labels = {x : 0 for [x, y] in labels.items() if y =='bonafide'}
binary_labels = {**one_labels, **zeros_labels}

### Define DataGenerator

In [21]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    
    def __init__(self, list_IDs, labels, root_path, batch_size=128, dim=(256, 256), n_channels=1,
             n_classes=2, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.root_path = root_path
        self.bonafide_IDs = [a for a in self.list_IDs if self.labels[a] == 0]
        self.bot_IDs = [a for a in self.list_IDs if self.labels[a] == 1]
        self.on_epoch_end()

        
    def on_epoch_end(self):
        self.bot_indexes = np.arange(len(self.bot_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.bot_indexes)
        self.bonafide_indexes = np.arange(len(self.bonafide_IDs))

            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
          # Store sample
          X[i] = np.expand_dims(np.abs(np.load(os.path.join(self.root_path, ID + '.npy'))), axis=2)

          # Store class
          y[i] = self.labels[ID]

        return X, tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)
                
    def __len__(self):
        return int(np.floor(len(self.bot_IDs) / (self.batch_size //2)))
    
    def __getitem__(self, index):
        
        # Generate indexes of the batch
        bot_indexes = self.bot_indexes[index*(self.batch_size // 2):(index+1)*(self.batch_size // 2)]
        bonafide_indexes = np.random.choice(self.bonafide_indexes, batch_size//2, replace=False)
        
        # Find list of IDs
        bot_list_IDs_temp = [self.bot_IDs[k] for k in bot_indexes]
        bonafide_list_IDs_temp = [self.bonafide_IDs[k] for k in bonafide_indexes]
        
        # Generate data
        X, y = self.__data_generation(bonafide_list_IDs_temp + bot_list_IDs_temp)

        return X, y

In [22]:
train_features_folder = '/nas/home/cborrelli/bot_speech/features/bicoherences/train_nfft_{}_hop_size_{}'.format(
    nfft, hop_size)

dev_features_folder = '/nas/home/cborrelli/bot_speech/features/bicoherences/dev_nfft_{}_hop_size_{}'.format(
    nfft, hop_size)

eval_features_folder = '/nas/home/cborrelli/bot_speech/features/bicoherences/eval_nfft_{}_hop_size_{}'.format(
    nfft, hop_size)

In [53]:
# Generators
batch_size = 32

training_generator = DataGenerator(partition['train'], binary_labels, train_features_folder, batch_size=batch_size)
validation_generator = DataGenerator(partition['dev'], binary_labels, dev_features_folder, batch_size=batch_size)

## Train tutorial CNN

In [54]:
logdir = "logs/scalars/model1_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=logdir)

In [61]:
num_classes = 2
epochs = 1

In [62]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [63]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=[256, 256, 1]))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 254, 254, 32)      320       
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 252, 252, 32)      9248      
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 124, 124, 32)      9248      
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 30, 30, 64)       

In [64]:
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['accuracy'])

reduce_lr = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=5, min_lr=0.001)

model.fit(training_generator, epochs=epochs,
                    validation_data=validation_generator,
                    use_multiprocessing=False)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1425 steps, validate for 1393 steps
1425/1425 [==============================] - 376s 264ms/step - loss: 0.4072 - accuracy: 0.8087 - val_loss: 0.5750 - val_accuracy: 0.7567


In [65]:
eval_generator = DataGenerator(partition['eval'], binary_labels, eval_features_folder, batch_size=batch_size)

In [66]:
score = model.evaluate(eval_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

  ...
    to  
  ['...']
3992/3992 [==============================] - 632s 158ms/step - loss: 0.7011 - accuracy: 0.6643
Test loss: 0.7011041119470744
Test accuracy: 0.6643208


## Train on train and dev

## ASV metrics

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score


In [ ]:



def compute_det_curve(target_scores, nontarget_scores):

    n_scores = target_scores.size + nontarget_scores.size
    all_scores = np.concatenate((target_scores, nontarget_scores))
    labels = np.concatenate((np.ones(target_scores.size), np.zeros(nontarget_scores.size)))

    # Sort labels based on scores
    indices = np.argsort(all_scores, kind='mergesort')
    labels = labels[indices]

    # Compute false rejection and false acceptance rates
    tar_trial_sums = np.cumsum(labels)
    nontarget_trial_sums = nontarget_scores.size - (np.arange(1, n_scores + 1) - tar_trial_sums)

    frr = np.concatenate((np.atleast_1d(0), tar_trial_sums / target_scores.size))  # false rejection rates
    far = np.concatenate((np.atleast_1d(1), nontarget_trial_sums / nontarget_scores.size))  # false acceptance rates
    thresholds = np.concatenate((np.atleast_1d(all_scores[indices[0]] - 0.001), all_scores[indices]))  # Thresholds are the sorted scores

    return frr, far, thresholds

def compute_eer(target_scores, nontarget_scores):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    frr, far, thresholds = compute_det_curve(target_scores, nontarget_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    return eer, thresholds[min_index]

## ASV scores on model1

In [ ]:
y_dev_hat = model.predict(np.expand_dims(norm_X_dev, axis=3))[:,0]


[fpr, tpr, th] = roc_curve(y_dev, y_dev_hat, pos_label=0)
roc_auc = 1 - roc_auc_score(y_dev, y_dev_hat)

acc = (tpr + (1-fpr)) / 2
best_th = th[np.where(acc == np.max(acc))]
print(best_th)
print(acc.max())

plt.plot(fpr, tpr, label='AUC={}'.format(roc_auc))
plt.plot([0, 1], [0, 1], ':k')
plt.plot([1, 0], [0, 1], ':k')
plt.axis([-0.01, 1, 0, 1.01])
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
target_score = y_dev_hat[np.where(y_dev==0)]
nontarget_score = y_dev_hat[np.where(y_dev==1)]

fnr = 1-tpr

plt.figure()
plt.plot(th, fnr)
plt.plot(th, fpr)
#plt.plot(th, np.abs(fnr-fpr))
plt.show()

compute_eer(target_score, nontarget_score)

## ASV scores on model2

In [ ]:
y_eval_hat = model2.predict(np.expand_dims(norm_X_eval, axis=3))[:,0]


[fpr, tpr, th] = roc_curve(y_eval, y_eval_hat, pos_label=0)
roc_auc = 1 - roc_auc_score(y_eval, y_eval_hat)

acc = (tpr + (1-fpr)) / 2
best_th = th[np.where(acc == np.max(acc))]
print(best_th)
print(acc.max())

plt.plot(fpr, tpr, label='AUC={}'.format(roc_auc))
plt.plot([0, 1], [0, 1], ':k')
plt.plot([1, 0], [0, 1], ':k')
plt.axis([-0.01, 1, 0, 1.01])
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
target_score = y_eval_hat[np.where(y_eval==0)]
nontarget_score = y_eval_hat[np.where(y_eval==1)]

fnr = 1-tpr

plt.figure()
plt.plot(th, fnr)
plt.plot(th, fpr)
#plt.plot(th, np.abs(fnr-fpr))
plt.show()

compute_eer(target_score, nontarget_score)